It's the extension of the work [SPINE](https://github.com/harsh19/SPINE/tree/master/code).

In [45]:
import torch
from torch import nn
import logging
import numpy as np
from sklearn.datasets import make_blobs
#logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.DEBUG)
import argparse
from random import shuffle

## Test Vectors
In interpretiblity results from word2vec, we were getting non sensible results by choosing the hyperparameter setting that have been discussed in the paper. 
To find the hyperparameters which might can work better we have created two vectors which were obtained from the original word2vec embedding file for the word Algebra and Mathematics. The idea was that to measure the cosine similarity between them during training. If the similarity is increasing between them during the training that entails that those hyperparameters are giving good results. Because what we have seen during training if the number of epochs are too much we loose so much information that transformation matrix almost becomes as identity matrix

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
algebra_word2vec=[0.141602 , -0.0927734 , -0.0108643 , 0.410156 , -0.0644531 , -0.0400391 , 0.445312 , -0.129883 , -0.191406 , 0.0668945 , 0.503906 , 0.136719 , -0.0563965 , -0.165039 , -0.0810547 , 0.131836 , 0.125977 , -0.300781 , 0.251953 , -0.0639648 , 0.130859 , 0.120605 , -0.0908203 , 0.166016 , -0.121094 , -0.291016 , -0.194336 , -0.277344 , -0.026123 , 0.149414 , -0.310547 , 0.304688 , -0.0766602 , 0.100586 , -0.0761719 , 0.208008 , 0.238281 , 0.0947266 , -0.0336914 , 0.394531 , -0.0908203 , 0.0349121 , 0.243164 , -0.212891 , 0.124512 , -0.0751953 , 0.0407715 , 0.0439453 , -0.205078 , 0.318359 , -0.519531 , -0.0927734 , -0.310547 , -0.166016 , 0.235352 , 0.488281 , 0.121582 , -0.0410156 , -0.228516 , -0.0737305 , 0.032959 , 0.207031 , 0.114258 , -0.100586 , 0.0551758 , -0.0795898 , -0.123047 , 0.188477 , 0.0272217 , -0.09375 , 0.0129395 , -0.277344 , -0.0192871 , 0.0101929 , -0.201172 , 0.133789 , 0.231445 , 0.118164 , 0.332031 , -0.10791 , -0.12793 , 0.10791 , -0.179688 , 0.117676 , 0.150391 , 0.233398 , 0.194336 , 0.707031 , -0.0522461 , -0.114258 , -0.0991211 , -0.683594 , -0.255859 , -0.112793 , -0.109863 , -0.111816 , 0.369141 , -0.0756836 , -0.105469 , -0.036377 , -0.00778198 , -0.231445 , -0.116211 , -0.148438 , -0.205078 , -0.230469 , -0.21582 , 0.117676 , 0.451172 , -0.172852 , 0.135742 , 0.0678711 , 0.134766 , -0.0231934 , 0.123535 , 0.078125 , 0.155273 , 0.267578 , 0.131836 , -0.169922 , 0.0117188 , 0.160156 , -0.0449219 , 0.0317383 , 0.208008 , -0.296875 , -0.128906 , 0.00738525 , -0.0703125 , 0.308594 , -0.144531 , -0.439453 , -0.257812 , -0.0634766 , 0.188477 , 0.259766 , 0.28125 , 0.0820312 , -0.160156 , 0.177734 , -0.257812 , -0.209961 , -0.330078 , -0.22168 , -0.255859 , 0.111328 , 0.144531 , -0.291016 , -0.0605469 , 0.15332 , 0.349609 , -0.137695 , -0.0123291 , 0.0510254 , -0.263672 , -0.330078 , -0.108887 , -0.10498 , -0.0446777 , 0.125 , -0.241211 , 0.046875 , 0.0213623 , -0.103027 , -0.335938 , 0.248047 , 0.554688 , -0.128906 , -0.0683594 , 0.206055 , 0.10498 , 0.182617 , -0.0424805 , -0.107422 , 0.367188 , 0.257812 , -0.148438 , -0.186523 , -0.0212402 , 0.228516 , -0.0932617 , -0.229492 , -0.0444336 , 0.0932617 , 0.539062 , 0.259766 , 0.111328 , -0.0737305 , 0.0358887 , 0.121582 , 0.00909424 , 0.0341797 , -0.244141 , 0.103516 , -0.244141 , 0.441406 , 0.235352 , -0.106934 , -0.0368652 , 0.00151062 , -0.306641 , 0.0539551 , 0.232422 , 0.00860596 , -0.225586 , -0.255859 , -0.024292 , -0.279297 , -0.139648 , 0.128906 , -0.203125 , 0.0913086 , 0.0275879 , -0.480469 , -0.243164 , 0.292969 , 0.188477 , -0.148438 , 0.168945 , -0.402344 , -0.217773 , 0.0932617 , 0.25 , -0.0981445 , 0.273438 , 0.133789 , -0.125977 , -0.0483398 , -0.0791016 , 0.0566406 , 0.12207 , 0.0427246 , -0.185547 , -0.152344 , 0.296875 , 0.193359 , 0.0546875 , 0.0166016 , 0.333984 , -0.00341797 , 0.228516 , 0.3125 , 0.195312 , -0.126953 , -0.261719 , -0.443359 , 0.0216064 , 0.0102539 , -0.550781 , -0.0549316 , -0.0581055 , -0.0869141 , 0.115234 , 0.546875 , -0.0786133 , -0.507812 , -0.208008 , -0.166016 , -0.0395508 , 0.570312 , 0.109863 , 0.0566406 , -0.326172 , -0.169922 , 0.0668945 , 0.168945 , 0.455078 , -0.269531 , -0.335938 , -0.0585938 , -0.135742 , 0.28125 , -0.158203 , 0.242188 , -0.277344 , 0.0688477 , 0.0878906 , -0.453125 , -0.3125 , -0.466797 , -0.0228271 , 0.200195 , -0.19043 , 0.132812 , -0.225586 , -0.136719 , -0.0116577 , 0.210938 , -0.0717773 , 0.402344 , -0.157227 , -0.164062 , 0.141602 , -0.0461426 , -0.139648 , 0.133789 , -0.154297 , 0.188477 , 0.304688 , 0.109863 ]
mathematics_word2vec=[0.235352 , -0.0422363 , 0.203125 , 0.0332031 , 0.150391 , 0.22168 , 0.248047 , -0.194336 , -0.0869141 , -0.0129395 , 0.28125 , 0.0786133 , -0.267578 , 0.0966797 , -0.118164 , 0.0020752 , -0.0130615 , -0.0524902 , 0.0996094 , -0.121582 , -0.0291748 , 0.0888672 , 0.0947266 , -0.144531 , 0.0678711 , -0.357422 , -0.0834961 , -0.123047 , -0.269531 , 0.0344238 , -0.115234 , -0.0493164 , 0.0698242 , -0.0444336 , -0.150391 , 0.0375977 , 0.0839844 , 0.351562 , 0.235352 , 0.189453 , -0.0148315 , 0.0203857 , 0.0307617 , 0.193359 , 0.117676 , 0.175781 , 0.0114746 , -0.0100708 , -0.201172 , 0.271484 , -0.349609 , -0.00637817 , 0.0262451 , -0.081543 , 0.191406 , 0.194336 , -0.0090332 , -0.273438 , -0.219727 , -0.259766 , 0.0437012 , -0.00701904 , 0.300781 , -0.259766 , 0.169922 , 0.0668945 , -0.21875 , 0.210938 , 0.059082 , -0.00830078 , -0.0380859 , -0.417969 , 0.251953 , -0.107422 , -0.0654297 , 0.102051 , 0.219727 , 0.275391 , 0.192383 , -0.163086 , -0.00540161 , 0.136719 , -0.00726318 , 0.106934 , 0.10791 , 0.0512695 , 0.168945 , 0.402344 , -0.102051 , -0.00582886 , 0.101074 , -0.386719 , -0.408203 , -0.0422363 , 0.0122681 , 0.118652 , 0.392578 , 0.0269775 , 0.0432129 , -0.0441895 , -0.00878906 , -0.036377 , -0.186523 , -0.231445 , 0.0090332 , -0.0952148 , -0.105957 , 0.140625 , 0.388672 , -0.048584 , 0.0839844 , -0.0771484 , 0.0279541 , 0.141602 , 0.239258 , 0.214844 , -0.081543 , -0.00939941 , 0.188477 , -0.00698853 , 0.0427246 , 0.257812 , -0.0373535 , -0.0314941 , -0.285156 , 0.116699 , -0.103027 , -0.0288086 , -0.166992 , 0.277344 , 0.125 , -0.460938 , -0.171875 , -0.359375 , 0.0854492 , 0.304688 , 0.0859375 , 0.0356445 , -0.326172 , 0.205078 , -0.0859375 , -0.129883 , -0.22168 , -0.0183105 , -0.394531 , 0.337891 , -0.0683594 , -0.0498047 , -0.220703 , 0.128906 , 0.105469 , 0.132812 , -0.145508 , 0.0854492 , -0.28125 , -0.269531 , 0.0229492 , -0.0419922 , -0.00387573 , -0.142578 , -0.145508 , -0.0255127 , -0.0168457 , 0.148438 , 0.137695 , 0.181641 , 0.378906 , -0.245117 , -0.0110474 , -0.078125 , -0.020752 , 0.164062 , -0.279297 , -0.0693359 , 0.324219 , 0.232422 , 0.0279541 , -0.231445 , 0.118652 , -0.3125 , -0.00778198 , 0.00933838 , -0.0405273 , 0.0358887 , 0.371094 , 0.147461 , 0.195312 , -0.0693359 , 0.182617 , 0.160156 , 0.0292969 , -0.0654297 , -0.115723 , 0.0375977 , -0.135742 , 0.214844 , 0.429688 , -0.169922 , -0.135742 , -0.00927734 , -0.335938 , -0.0185547 , 0.231445 , -0.0727539 , -0.251953 , -0.273438 , 0.225586 , -0.115234 , -0.324219 , 0.02771 , -0.0883789 , -0.0437012 , 0.135742 , -0.0461426 , -0.0405273 , 0.124023 , 0.126953 , 0.116211 , 0.0174561 , -0.0383301 , -0.197266 , -0.174805 , 0.242188 , -0.0844727 , 0.0354004 , 0.143555 , 0.10498 , -0.0634766 , -0.0756836 , 0.0908203 , 0.0996094 , 0.0717773 , -0.0256348 , 0.132812 , 0.146484 , 0.0297852 , 0.193359 , -0.173828 , 0.208984 , 0.0537109 , 0.0147705 , 0.108887 , -0.117188 , 0.0820312 , -0.203125 , -0.298828 , 0.18457 , 0.0644531 , -0.376953 , 0.0114136 , -0.129883 , -0.141602 , 0.0664062 , 0.431641 , 0.0913086 , -0.182617 , -0.164062 , 0.155273 , -0.0140381 , 0.322266 , 0.423828 , -0.00640869 , -0.154297 , 0.0181885 , -0.0546875 , 0.241211 , 0.240234 , -0.15332 , -0.277344 , -0.0830078 , 0.0153809 , -0.00393677 , -0.384766 , -0.166992 , -0.0280762 , -0.0202637 , 0.0874023 , -0.185547 , -0.212891 , 0.0112915 , -0.00726318 , 0.157227 , -0.078125 , 0.03125 , -0.0874023 , -0.116211 , 0.081543 , 0.0893555 , -0.00546265 , 0.445312 , -0.238281 , 0.15332 , -0.186523 , -0.120605 , 0.0527344 , -0.024292 , 0.0473633 , -0.0380859 , 0.337891 , -0.0639648 ]
algebra_word2vec=torch.tensor(algebra_word2vec)
algebra_word2vec=algebra_word2vec.to(device)
mathematics_word2vec=torch.tensor(mathematics_word2vec)
mathematics_word2vec=mathematics_word2vec.to(device)
print(mathematics_word2vec.size(0))
print(algebra_word2vec.shape)


300
torch.Size([300])


# Dataset Downloading
The data can be downloaded from this [link](https://drive.google.com/drive/folders/1ksVcWDADmnp0Cl5kezjHqTg3Jnh8q031).

In [47]:
# Downloading Data
def files(FILEID,FILENAME):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O '{FILENAME}'
!mkdir data 
!mkdir output
file_name_train_word2vec='data/word2vec_original_15k_300d_train.txt'
file_name_train_glovec='data/glove_original_15k_300d_train.txt'
file_name_result_word2vec='data/SPINE_word2vec.txt'
files('1GqxXcA7EOkzlfA8oYIXLxMa72bsUe_xY','data/word2vec_original_15k_300d_train.txt')
files('1vwfQ8v4AhoYV_mLh5qcO8-iGLW8CwWJT','data/word2vec_original_3k_300d_val.txt')
files('1vhvUhMHY8eBPsetYBTicNKIN3Jxup6Ji','data/glove_original_15k_300d_train.txt')
files('1O8g4xXkqCJfeez3NfHEA0Zsl7dP1wTww',file_name_result_word2vec)


mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘output’: File exists
--2021-07-12 11:38:52--  https://docs.google.com/uc?export=download&id=1GqxXcA7EOkzlfA8oYIXLxMa72bsUe_xY
Resolving docs.google.com (docs.google.com)... 74.125.128.102, 74.125.128.101, 74.125.128.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.128.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-b4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/asv6bpia9ffs5kgd5dm2avkcp3p3g337/1626089925000/14542752466560403264/*/1GqxXcA7EOkzlfA8oYIXLxMa72bsUe_xY?e=download [following]
--2021-07-12 11:38:53--  https://doc-10-b4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/asv6bpia9ffs5kgd5dm2avkcp3p3g337/1626089925000/14542752466560403264/*/1GqxXcA7EOkzlfA8oYIXLxMa72bsUe_xY?e=download
Resolving doc-10-b4-docs.googleusercontent.com (doc-10-b4-docs.googleusercontent.com)..

In [48]:
# downloaded files
!ls data

glove_original_15k_300d_train.txt  word2vec_original_15k_300d_train.txt
SPINE_word2vec.txt		   word2vec_original_3k_300d_val.txt


# Visualization Function
The visualization function is provided in the github reposity which is based on the word intrusion test

In [73]:
from multiprocessing import Queue
from queue import PriorityQueue
from numpy import linalg as LA
from scipy import stats
import numpy as np
import argparse
import time
import sys
import os
from tqdm import tqdm

class Visualize:
  def __init__(self,filename):
    self.top_k_words = []
    self.zeros=0.0
    self.threshold = 0.001
    self.total = None
    self.width = 5
    self.vectors = None
    self.fileName = filename
    self.load_vectors()
  def load_vectors(self):
    
    self.vectors = {}
    self.zeros = 0.0
    f = open(self.fileName, 'r')
    lines = f.readlines()
    f.close()
    dimension = len(lines[0].split()) - 1
    self.top_k_words = [ [] for i in range(dimension)]
    c = 0
    for line in tqdm(lines):
        start = time.time()
        words = line.strip().split()
        self.vectors[words[0]] = [abs(float(i)) for i in words[1:]]
        self.h_dim = len(words[1:])
        c += 1
        vector = self.vectors[words[0]]
        for i, val in enumerate(vector):
          temp = self.top_k_words[i]
          if len(temp) < self.width:
            temp.append((val,words[0]))
          else:
            check = temp[-1]
            if check[0] < val:
              temp[-1] = (val, words[0])
          self.top_k_words[i] = sorted(temp, reverse=True)
        self.zeros += sum([1 for i in self.vectors[words[0]] if i < self.threshold])
    print ("Sparsity =", 100. * self.zeros/(len(lines)*dimension))
    self.total = len(self.vectors)
    print ('done loading vectors')
  def load_top_dimensions(self,k):
    self.top_k_words
    return
    self.dimensions = len(self.vectors[self.vectors.keys()[0]])
    for i in range(self.dimensions):
      temp = []
      while self.top_k_words[i].qsize() > 0:
        temp.append(self.top_k_words.get_nowait()[1])
      self.top_k_words[i] = temp
    print ('loaded top dimensions')
  def find_top_participating_dimensions(self,word, k):
    if word not in self.vectors:
      print ('word not found')
      return []
    temp = [(j, i) for i, j in enumerate(self.vectors[word])]
    answer = []
    print (" -----------------------------------------------------")
    print ("Word of interest = " , word)
    for i, j in sorted(temp, reverse=True)[:k]:
      print ("The contribution of the word '%s' in dimension %d = %f" %(word, j, i))
      print ('Following are the top words in dimension', j, 'along with their contributions')
      print (self.top_k_words[j])
      answer.append([k[1] for k in self.top_k_words[j]])
    return

# Before training
Let's see the interpretibility of both representation before applying SPINE

In [74]:
bt_word2vec=Visualize(file_name_train_word2vec)
bt_glove=Visualize(file_name_train_glovec)

  2%|▏         | 246/15000 [00:00<00:06, 2454.41it/s]

Sparsity = 0.4829555555555556
done loading vectors


100%|██████████| 15000/15000 [00:05<00:00, 2640.61it/s]


Sparsity = 0.22595555555555555
done loading vectors


## Before training: Ouput of word2vec 

In [76]:
bt_word2vec.find_top_participating_dimensions("mathematics",3)

 -----------------------------------------------------
Word of interest =  mathematics
The contribution of the word 'mathematics' in dimension 131 = 0.460938
Following are the top words in dimension 131 along with their contributions
[(0.785156, 'leukemia'), (0.714844, 'enterprises'), (0.703125, 'wingspan'), (0.664062, 'info'), (0.644531, 'booker')]
The contribution of the word 'mathematics' in dimension 289 = 0.445312
Following are the top words in dimension 289 along with their contributions
[(0.84375, 'ore'), (0.804688, 'greens'), (0.769531, 'badminton'), (0.765625, 'hymns'), (0.761719, 'clay')]
The contribution of the word 'mathematics' in dimension 253 = 0.431641
Following are the top words in dimension 253 along with their contributions
[(0.839844, 'asylum'), (0.773438, 'intercepted'), (0.769531, 'skater'), (0.757812, 'rb'), (0.753906, 'flats')]


## Before training: Ouput of glove

In [77]:
bt_glove.find_top_participating_dimensions("mathematics",3)

 -----------------------------------------------------
Word of interest =  mathematics
The contribution of the word 'mathematics' in dimension 9 = 1.252700
Following are the top words in dimension 9 along with their contributions
[(3.0241, 'intelligence'), (2.9325, 'government'), (2.7918, 'foreign'), (2.7698, 'security'), (2.7656, 'cooperation')]
The contribution of the word 'mathematics' in dimension 144 = 1.195400
Following are the top words in dimension 144 along with their contributions
[(1.8367, 'kashmir'), (1.6063, 'algorithms'), (1.5863, 'heat'), (1.5809, 'computational'), (1.546, 'unmanned')]
The contribution of the word 'mathematics' in dimension 177 = 0.951220
Following are the top words in dimension 177 along with their contributions
[(1.5781, 'robes'), (1.5557, 'tito'), (1.4902, 'aviation'), (1.4356, 'backward'), (1.3881, 'dioceses')]


# Datahandler class

In [78]:
############################################

class DataHandler:

	def __init__(self):
		pass

	def loadData(self, filename):
		logging.info("Loading data from " + filename )
		#limit = 1000 ## for debnugging. TODO: remove this
		lines = open(filename).readlines()#[:limit] 
		self.data = []
		self.words = []
		for line in lines:
			tokens = line.strip().split()
			self.words.append(tokens[0])
			self.data.append([float(i) for i in tokens[1:]])
		self.data = np.array(self.data)
		logging.info("Loaded data. #shape = " + str(self.data.shape) )
		logging.info(" #words = %d " %(len(self.words)) )
		self.data_size = self.data.shape[0]
		self.inp_dim = self.data.shape[1]
		self.original_data = self.data[:]
		logging.debug("original_data[0][0:5] = " + str(self.original_data[0][0:5]))

	def getWordsList(self):
		return self.words

	def getDataShape(self):
		return self.data.shape

	def resetDataOrder(self):
		self.data = self.original_data[:]
		logging.debug("original_data[0][0:5] = " + str(self.original_data[0][0:5]))

	def getNumberOfBatches(self, batch_size):
		return int(( self.data_size + batch_size - 1 ) / batch_size)

	def getBatch(self, i, batch_size, noise_level, denoising):
		batch_y = self.data[i*batch_size:min((i+1)*batch_size, self.data_size)]
		batch_x = batch_y
		if denoising:
			batch_x = batch_y + get_noise_features(batch_y.shape[0], self.inp_dim, noise_level)
		return batch_x, batch_y

	def shuffleTrain(self):
		indices = np.arange(self.data_size)
		np.random.shuffle(indices)
		self.data = self.data[indices]

############################################

def compute_sparsity(X):
	non_zeros = 1. * np.count_nonzero(X)
	total = X.size
	sparsity = 100. * (1 - (non_zeros)/total)
	return sparsity

def dump_vectors(X, outfile, words):
	print ("shape", X.shape)
	assert len(X) == len(words) #TODO print error statement
	fw = open(outfile, 'w')
	for i in range(len(words)):
		fw.write(words[i] + " ")
		for j in X[i]:
			fw.write(str(j) + " ")
		fw.write("\n")
	fw.close()

def get_noise_features(n_samples, n_features, noise_amount):
	noise_x,  _ =  make_blobs(n_samples=n_samples, n_features=n_features, 
			cluster_std=noise_amount,
			centers=np.array([np.zeros(n_features)]))
	return noise_x

# Model

In [79]:
###### Solver class ######
class Solver:

  def __init__(self, params):

		# Build data handler
    self.data_handler = DataHandler()
    self.data_handler.loadData(params['input'])
    params['inp_dim'] = self.data_handler.getDataShape()[1]
    logging.info("="*41)
    # Build model
    self.model = SPINEModel(params)
    self.dtype = torch.FloatTensor
    use_cuda = torch.cuda.is_available()
    if use_cuda:
      self.model.cuda()
      self.dtype = torch.cuda.FloatTensor
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.1)
    logging.info("="*41)

  def train(self, params):
    num_epochs, batch_size = params['num_epochs'], params['batch_size'],
    optimizer = self.optimizer
    dtype = self.dtype
    for iteration in range(num_epochs):
      self.data_handler.shuffleTrain()
      num_batches = self.data_handler.getNumberOfBatches(batch_size)
      epoch_losses = np.zeros(5) # rl, asl, psl, total, test_loss
      exceed=False
      for batch_idx in range(num_batches):
        optimizer.zero_grad()
        batch_x, batch_y = self.data_handler.getBatch(batch_idx, batch_size, params['noise_level'], params['denoising'] )
        batch_x = torch.from_numpy(batch_x).type(dtype)
        batch_y = torch.from_numpy(batch_y).type(dtype)
        out, h, loss, loss_terms,test_loss = self.model(batch_x, batch_y)
        reconstruction_loss, psl_loss, asl_loss = loss_terms
        loss.backward()
        optimizer.step()
        epoch_losses[0]+=reconstruction_loss.item()
        epoch_losses[1]+=asl_loss.item()
        epoch_losses[2]+=psl_loss.item()
        epoch_losses[3]+=loss.item()
        epoch_losses[4]+=test_loss.item()
      print("After epoch %r, Reconstruction Loss = %.4f, ASL = %.4f,"\
          "PSL = %.4f, and total = %.4f"
          %(iteration+1, epoch_losses[0], epoch_losses[1], epoch_losses[2], epoch_losses[3]) )
      print(f'cosine_sim_w2vec={epoch_losses[4]:.4f}')
      sparsity_loss=compute_sparsity(h.cpu().data.numpy())
      logging.info("After epoch %r, Sparsity = %.1f"%(iteration+1, sparsity_loss))
      if sparsity_loss<.95:
        exceed=True
        break
      


		

  def getSpineEmbeddings(self, batch_size, params):
    ret = []
    self.data_handler.resetDataOrder()
    num_batches = self.data_handler.getNumberOfBatches(batch_size)
    for batch_idx in range(num_batches):
      batch_x, batch_y = self.data_handler.getBatch(batch_idx, batch_size, params['noise_level'], params['denoising'] )
      batch_x = torch.from_numpy(batch_x).type(self.dtype)
      batch_y = torch.from_numpy(batch_y).type(self.dtype)
      _, h, _, _,_ = self.model(batch_x, batch_y)
      ret.extend(h.cpu().data.numpy())
    return np.array(ret)

  def getWordsList(self):
    return self.data_handler.getWordsList()

##########################################################################
#############################Model########################################
##########################################################################


# Model

logging.basicConfig(level=logging.INFO)
cos_sim = nn.CosineSimilarity(dim=0, eps=1e-6)


class SPINEModel(torch.nn.Module):

  def __init__(self, params):
    super(SPINEModel, self).__init__()
    # params
    self.inp_dim = params['inp_dim']
    self.hdim = params['hdim']
    self.noise_level = params['noise_level']
    self.getReconstructionLoss = nn.MSELoss()
    self.rho_star = 1.0 - params['sparsity']
		
    # autoencoder
    logging.info("Building model ")
    self.linear1 = nn.Linear(self.inp_dim, self.hdim)
    self.linear2 = nn.Linear(self.hdim, self.inp_dim)
		
  def modelCompute(self,x):
    linear1_out = self.linear1(x)
    h = linear1_out.clamp(min=0, max=1)
    out = self.linear2(h)
    return out
  def testSimilarity(self):
    rep1=self.modelCompute(algebra_word2vec)
    rep2=self.modelCompute(mathematics_word2vec)
    return cos_sim(rep1,rep2)

   
  def forward(self, batch_x, batch_y):	
    # forward
    batch_size = batch_x.size(0)
    linear1_out = self.linear1(batch_x)
    # capped relu
    h = linear1_out.clamp(min=0, max=1) 
    out = self.linear2(h)

    # different terms of the loss
    reconstruction_loss = self.getReconstructionLoss(out, batch_y) # reconstruction loss
    psl_loss = self._getPSLLoss(h, batch_size) 		# partial sparsity loss
    asl_loss = self._getASLLoss(h)    	# average sparsity loss
    total_loss = reconstruction_loss + psl_loss + asl_loss
    test_loss = self.testSimilarity()
    return out, h, total_loss, [reconstruction_loss,psl_loss, asl_loss],test_loss


  def _getPSLLoss(self,h, batch_size):
    return torch.sum(h*(1-h))/ (batch_size * self.hdim)


  def _getASLLoss(self, h):
    temp = torch.mean(h, dim=0) - self.rho_star
    temp = temp.clamp(min=0)
    return torch.sum(temp * temp) / self.hdim





# Main function

In [80]:
#########################################################
# main program
def main(hdim=1000,
                  denoising=False,
                  noise_level=0.2,
                  num_epochs=100,
                  batch_size=64,
                  sparsity=0.85,
                  fileName='word2vec_original_15k_300d_train.txt'):
  params=dict()
  # params['inp_dim']=2000
  params['hdim']=hdim
  params['noise_level']=noise_level
  params['denoising']=denoising
  params['sparsity']=sparsity
  params['num_epochs']=num_epochs
  params['batch_size']=batch_size
  params['input']='data/'+fileName
  params['output']='output/'+fileName
  solver = Solver(params)
  solver.train( params )
  # dumping the final vectors
  logging.info("Dumping the final SPine embeddings")
  output_path = params['output'] + ".spine"
  final_batch_size = 512
  spine_embeddings = solver.getSpineEmbeddings(final_batch_size, params)
  dump_vectors(spine_embeddings, output_path, solver.getWordsList())
  
# For testing
# main(num_epochs=2,denoising=True)

# Glove Training

In [ ]:
# glove parameters
epochs=4000
filename=file_name_train_glovec.replace("data/","")
hdim=1000
denoising=True
noise_level=0.4
batch_size=64
sparsity=0.85
print('##########################################################')
print('################## Glove File generation #################')
print('##########################################################')
print('##########################################################')
main(num_epochs=epochs,denoising=denoising,noise_level=noise_level,
     hdim=hdim,batch_size=batch_size,sparsity=sparsity,
     fileName=filename)

INFO:root:Loading data from data/glove_original_15k_300d_train.txt


##########################################################
################## Glove File generation #################
##########################################################
##########################################################


INFO:root:Loaded data. #shape = (15000, 300)
INFO:root: #words = 15000 
INFO:root:=========================================
INFO:root:Building model 
INFO:root:=========================================


After epoch 1, Reconstruction Loss = 33.5532, ASL = 0.0449,PSL = 17.5195, and total = 51.1176
After epoch 2, Reconstruction Loss = 32.0010, ASL = 0.0251,PSL = 16.5910, and total = 48.6172
After epoch 3, Reconstruction Loss = 31.0752, ASL = 0.0145,PSL = 15.7471, and total = 46.8368
After epoch 4, Reconstruction Loss = 30.2911, ASL = 0.0085,PSL = 14.9930, and total = 45.2926
After epoch 5, Reconstruction Loss = 29.6417, ASL = 0.0053,PSL = 14.2861, and total = 43.9331
After epoch 6, Reconstruction Loss = 29.0882, ASL = 0.0036,PSL = 13.6620, and total = 42.7538
After epoch 7, Reconstruction Loss = 28.5937, ASL = 0.0029,PSL = 13.0883, and total = 41.6849
After epoch 8, Reconstruction Loss = 28.1865, ASL = 0.0023,PSL = 12.5645, and total = 40.7533
After epoch 9, Reconstruction Loss = 27.7951, ASL = 0.0023,PSL = 12.1134, and total = 39.9107
After epoch 10, Reconstruction Loss = 27.4530, ASL = 0.0023,PSL = 11.6913, and total = 39.1466
After epoch 11, Reconstruction Loss = 27.1534, ASL = 0.0024

INFO:root:Dumping the final SPine embeddings


After epoch 4000, Reconstruction Loss = 12.0975, ASL = 0.0080,PSL = 1.8759, and total = 13.9815
shape (15000, 1000)


# Word2vec Training

In [105]:
# word2vec parameters
epochs=600
filename=file_name_train_word2vec.replace("data/","")
hdim=1000
denoising=True
noise_level=0.6
batch_size=512
sparsity=0.85
print('##########################################################')
print('################# word2vec File generation ###############')
print('##########################################################')
print('##########################################################')
main(num_epochs=epochs,denoising=denoising,noise_level=noise_level,
     hdim=hdim,batch_size=batch_size,sparsity=sparsity,
     fileName=filename)

INFO:root:Loading data from data/word2vec_original_15k_300d_train.txt


##########################################################
################# word2vec File generation ###############
##########################################################
##########################################################


INFO:root:Loaded data. #shape = (15000, 300)
INFO:root: #words = 15000 
INFO:root:=========================================
INFO:root:Building model 
INFO:root:=========================================
INFO:root:After epoch 1, Sparsity = 50.1


After epoch 1, Reconstruction Loss = 1.4971, ASL = 0.0060,PSL = 2.3811, and total = 3.8843
cosine_sim_w2vec=23.4956


INFO:root:After epoch 2, Sparsity = 50.2


...

After epoch 598, Reconstruction Loss = 0.8687, ASL = 0.0000,PSL = 0.2588, and total = 1.1275
cosine_sim_w2vec=30.0000


INFO:root:After epoch 599, Sparsity = 91.7


After epoch 599, Reconstruction Loss = 0.8686, ASL = 0.0000,PSL = 0.2556, and total = 1.1242
cosine_sim_w2vec=30.0000


INFO:root:After epoch 600, Sparsity = 91.4
INFO:root:Dumping the final SPine embeddings


After epoch 600, Reconstruction Loss = 0.8679, ASL = 0.0000,PSL = 0.2572, and total = 1.1251
cosine_sim_w2vec=30.0000
shape (15000, 1000)


# Visualization of outputs

In [93]:
# in case you want to load file from your google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Word2vec visualization

In [106]:
# word2vec visualization Loading file
fileAreadyAvailable=False
filename_output=""
if fileAreadyAvailable:
  PATH='gdrive/MyDrive/DL_project_ouputFiles/'
  filename_output=PATH+file_name_train_word2vec.replace("data/","")+".spine"
else:
  PATH = "output/"
  filename_output=PATH+file_name_train_word2vec.replace("data/","")+".spine"
  # filename_ouput=file_name_result_word2vec
at_word2vec=Visualize(filename_output)

100%|██████████| 15000/15000 [00:21<00:00, 693.73it/s]


Sparsity = 91.45378
done loading vectors


In [107]:
print('##########################################################')
print('################## word2vec File testing #################')
print('##########################################################')
print('##########################################################')
at_word2vec.find_top_participating_dimensions("mathematics",3)

##########################################################
################## word2vec File testing #################
##########################################################
##########################################################
 -----------------------------------------------------
Word of interest =  mathematics
The contribution of the word 'mathematics' in dimension 817 = 0.506257
Following are the top words in dimension 817 along with their contributions
[(0.6711867, 'performance'), (0.6681726, 'amounts'), (0.63069785, 'choreography'), (0.59249234, 'permanent'), (0.5533337, 'purchased')]
The contribution of the word 'mathematics' in dimension 730 = 0.503275
Following are the top words in dimension 730 along with their contributions
[(0.6962829, 'injected'), (0.6912365, 'departure'), (0.6447733, 'haas'), (0.58231205, 'found'), (0.5625863, 'shocks')]
The contribution of the word 'mathematics' in dimension 412 = 0.485866
Following are the top words in dimension 412 along with t

If you want to copy the ouptput file to your google drive lcoation

In [110]:
!cp /content/output/word2vec_original_15k_300d_train.txt.spine /content/gdrive/MyDrive/DL_project_ouputFiles/

## Glove results 

In [108]:
# Glove visualization
# word2vec visualization Loading file
fileAreadyAvailable=True
filename_output=""
if fileAreadyAvailable:
  PATH='gdrive/MyDrive/DL_project_ouputFiles/'
  filename_output=PATH+file_name_train_glovec.replace("data/","")+".spine"
else:
  PATH = "output/"
  filename_output=PATH+file_name_train_glovec.replace("data/","")+".spine"
  # filename_ouput=file_name_result_word2vec
at_glove=Visualize(filename_output)

100%|██████████| 15000/15000 [00:27<00:00, 551.34it/s]


Sparsity = 93.5858
done loading vectors


In [109]:
print('##########################################################')
print('#################### Glove File testing ##################')
print('##########################################################')
print('##########################################################')
at_glove.find_top_participating_dimensions("mathematics",3)

##########################################################
#################### Glove File testing ##################
##########################################################
##########################################################
 -----------------------------------------------------
Word of interest =  mathematics
The contribution of the word 'mathematics' in dimension 933 = 1.000000
Following are the top words in dimension 933 along with their contributions
[(1.0, 'theory'), (1.0, 'related'), (1.0, 'questions'), (1.0, 'laws'), (1.0, 'items')]
The contribution of the word 'mathematics' in dimension 696 = 1.000000
Following are the top words in dimension 696 along with their contributions
[(1.0, 'university'), (1.0, 'science'), (1.0, 'professor'), (1.0, 'politics'), (1.0, 'law')]
The contribution of the word 'mathematics' in dimension 641 = 1.000000
Following are the top words in dimension 641 along with their contributions
[(1.0, 'university'), (1.0, 'students'), (1.0, 'school')

If you want to copy the ouput file into your google drive

In [ ]:
!cp /content/output/glove_original_15k_300d_train.txt.spine /content/gdrive/MyDrive/DL_project_ouputFiles/